In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

import multiprocess
# with multiprocess.Pool(processes=8) as pool:
#   for i in pool.imap_unordered(f, a):
#     print(i)

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('17')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [2]:
lines_temp, groups_temp = linesplit("""
####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
""".strip())

In [3]:
lines, groups = linesplit("""
>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
""".strip())

In [38]:
lines, groups = linesplit(raw_input, verbose=True)

1 lines
1 groups
>>>
>>>><<>>>><<><<<>>><<<>><<>>><<>>>><<<>>><<<>><<<<>>>><<<<>>>><<<>>>><<<<><><>>><<<<>><>>>><<<><<<>>...


In [4]:
minos = []

for group in groups_temp:
  ys = len(group)
  xs = len(group[0])
  mino = []
  
  for i in range(len(group)):
    for j in range(len(group[0])):
      if group[i][j] == '#':
        mino.append(Point(j, ys - i - 1))
  
  minos.append(mino)

minos

[[[+0, +0], [+1, +0], [+2, +0], [+3, +0]],
 [[+1, +2], [+0, +1], [+1, +1], [+2, +1], [+1, +0]],
 [[+2, +2], [+2, +1], [+0, +0], [+1, +0], [+2, +0]],
 [[+0, +3], [+0, +2], [+0, +1], [+0, +0]],
 [[+0, +1], [+1, +1], [+0, +0], [+1, +0]]]

In [5]:
width = 7

In [6]:
jets = lines[0]

mino_index = 0
jet_index = 0

stored = set()

def moved(points, delta):
  return [i + delta for i in points]

for i in range(2022):
  a = list(minos[mino_index])
  mino_index = (mino_index + 1) % len(minos)
  
  base_x = 2
  if stored:
    base_y = max([i.y for i in stored]) + 4
  else:
    base_y = 3
  
  a = [i + Point(base_x, base_y) for i in a]
  
  while True:
    aa = moved(a, LEFT if jets[jet_index] == '<' else RIGHT)
    jet_index = (jet_index + 1) % len(jets)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      pass
    else:
      a = aa
      
      
    aa = moved(a, DOWN)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      stored |= set(a)
      
      break
    else:
      a = aa

max([i.y for i in stored]) + 1

3068

In [8]:
jets = lines[0]

mino_index = 0
jet_index = 0

stored = set()

def moved(points, delta):
  return [i + delta for i in points]

memo = {}

maxdepth = 0

for i in tqdm(range(202200)):
  a = list(minos[mino_index])
  mino_index = (mino_index + 1) % len(minos)
  
  base_x = 2
  if stored:
    base_y = max([i.y for i in stored]) + 4
  else:
    base_y = 3
    
    
  bottom = []
  for x in range(0, 7):
    for y in range(base_y - 10, base_y):
      p = Point(x, y)
      if p in stored:
        bottom.append(p)
  
  state = (
    mino_index,
    jet_index,
    tuple(bottom)
  )
  
  if state in memo:
    print(i)
    print(memo[state])
    print(state)
    break
  else:
    memo[state] = i
  
  a = [i + Point(base_x, base_y) for i in a]
  
  while True:
    aa = moved(a, LEFT if jets[jet_index] == '<' else RIGHT)
    jet_index = (jet_index + 1) % len(jets)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      pass
    else:
      a = aa
      
      
    aa = moved(a, DOWN)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      stored |= set(a)
      maxdepth = max(maxdepth, base_y - min([i.y for i in a]))
      
      break
    else:
      a = aa

max([i.y for i in stored]) + 1

  0%|          | 0/202200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
maxdepth

12